In [ ]:
import os
import time
import datetime
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader, TensorDataset, SequentialSampler, RandomSampler
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

from utils_data import Vocabulary, Vectorizer, HeadQA, HeadQA_IR, clean_words, parse_dataset, parse_ir_dataset, random_oversamplig
from training import train, validate, evaluate

import transformers
from transformers.optimization import AdamW
from transformers import BertForSequenceClassification, BertConfig, BertTokenizer, BertModel, BertForMaskedLM

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from datasets import load_dataset

data_es = load_dataset('head_qa', 'es' )

In [ ]:
training, validation, testing = data_es['train'], data_es['validation'], data_es['test']

In [ ]:
training[0]

In [ ]:
training_instances = parse_ir_dataset(training)
validation_instances = parse_ir_dataset(validation)
testing_instances = parse_ir_dataset(testing)

oversampled_training = random_oversamplig(training_instances)

In [ ]:
oversampled_training[0]

In [ ]:
vectorizer = Vectorizer.vectorize_ir_dataset(oversampled_training)

In [ ]:
vocab = vectorizer.sentence_vocab
label_vocab = vectorizer.label_vocab

In [ ]:
trainset = HeadQA_IR(instances=training_instances, vectorizer=vectorizer, right_padding=False, max_length=15)
validset = HeadQA_IR(instances=validation_instances, vectorizer=vectorizer, right_padding=False, max_length=15)
testset = HeadQA_IR(instances=testing_instances, vectorizer=vectorizer, right_padding=False, max_length=15)

In [ ]:
trainset.instances[0]

In [ ]:
batch_size = 32
train_dt = DataLoader(trainset, batch_size=batch_size,drop_last=True)
valid_dt = DataLoader(validset, batch_size=batch_size,drop_last=True)
test_dt = DataLoader(testset, batch_size=batch_size,drop_last=True)

In [ ]:
class LSTM_QA(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim, x_size, n_classes, embedding_dim=300): 
        super(BasicLSTM, self).__init__()
        self.lstm_0 = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.lstm_1 = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        #nn.CosineSimilarity
        self.linear = nn.Linear(hidden_dim, n_classes)
        self.dropout = nn.Dropout(0.5)
        self.n_classes = n_classes
        
        
    def forward(self, x_0, x_1):
        out_0, (ht_0, ct_0) = self.lstm_0(x_0)
        out_1, (ht_1, ct_1) = self.lstm_0(x_1)
        print(out_0.shape, out_1.shape)
        print(ht_0.shape, ht_1.shape)
        print(ct_0,shape, ct_1.shape)
        return    

In [ ]:
def get_optimizer(model, lr=0.01, wd=0.0):
    return torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)